In [21]:
import pandas as pd
import psycopg2
import numpy as np
import os
import sql as gd
import psycopg2 # Needed to get database errors when uploading dataframe
import sqlalchemy as sqla

In [22]:
population_2000_2020=pd.read_csv("data/12411-0014_2000_2020.csv",header=[0,1] 
  )

population_2000_2020

Unnamed: 0_level_0                      Unnamed: 1_level_0  \
      Unnamed: 0_level_1 Unnamed: 0_level_1.1 Unnamed: 1_level_1   
0                      0           2000-12-31  Baden-Württemberg   
1                      1           2000-12-31  Baden-Württemberg   
2                      2           2000-12-31  Baden-Württemberg   
3                      3           2000-12-31  Baden-Württemberg   
4                      4           2000-12-31  Baden-Württemberg   
...                  ...                  ...                ...   
30907               1467           2020-12-31          Thüringen   
30908               1468           2020-12-31          Thüringen   
30909               1469           2020-12-31          Thüringen   
30910               1470           2020-12-31          Thüringen   
30911               1471           2020-12-31          Thüringen   

      Unnamed: 2_level_0 Germans                   Foreigners                 \
      Unnamed: 2_level_1    Male   Female    Total       Male Female   Total   
0           under 1 year   52648    50202   102850       1748   1706    3454   
1                 1 year   48282    45273    93555       7981   7411   15392   
2                2 years   49526    46826    96352       8128   7813   15941   
3                3 years   51550    48941   100491       8509   8026   16535   
4                4 years   50966    48167    99133       8063   7889   15952   
...                  ...     ...      ...      ...        ...    ...     ...   
30907           87 years    2973     5387     8360          9      8      17   
30908           88 years    2468     4925     7393         11     11      22   
30909           89 years    2079     4335     6414         11      3      14   
30910  90 years and over    5812    17402    23214         24     45      69   
30911              Total  984351  1021846  2006197      64861  49179  114040   

         Total                    
          Male   Female    Total  
0        54396    51908   106304  
1        56263    52684   108947  
2        57654    54639   112293  
3        60059    56967   117026  
4        59029    56056   115085  
...        ...      ...      ...  
30907     2982     5395     8377  
30908     2479     4936     7415  
30909     2090     4338     6428  
30910     5836    17447    23283  
30911  1049212  1071025  2120237  

[30912 rows x 13 columns]

In [24]:
population_2000_2020.columns=["num", "date", "state", "age", "german_male", "german_female",
 "german_total", "foreigner_male", "foreigner_female", "foreigner_total",
  "total_male", "total_female", "total"]
population_2000_2020

,num,date,state,age,german_male,german_female,german_total,foreigner_male,foreigner_female,foreigner_total,total_male,total_female,total
0,0,2000-12-31,Baden-Württemberg,under 1 year,52648,50202,102850,1748,1706,3454,54396,51908,106304
1,1,2000-12-31,Baden-Württemberg,1 year,48282,45273,93555,7981,7411,15392,56263,52684,108947
2,2,2000-12-31,Baden-Württemberg,2 years,49526,46826,96352,8128,7813,15941,57654,54639,112293
3,3,2000-12-31,Baden-Württemberg,3 years,51550,48941,100491,8509,8026,16535,60059,56967,117026
4,4,2000-12-31,Baden-Württemberg,4 years,50966,48167,99133,8063,7889,15952,59029,56056,115085
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30907,1467,2020-12-31,Thüringen,87 years,2973,5387,8360,9,8,17,2982,5395,8377
30908,1468,2020-12-31,Thüringen,88 years,2468,4925,7393,11,11,22,2479,4936,7415
30909,1469,2020-12-31,Thüringen,89 years,2079,4335,6414,11,3,14,2090,4338,6428
30910,1470,2020-12-31,Thüringen,90 years and over,5812,17402,23214,24,45,69,5836,17447,23283


In [ ]:
population_2000_2020 = population_2000_2020.drop('num', 1)
population_2000_2020

In [28]:
population_2000_2020 = population_2000_2020[population_2000_2020.age != "Total"]

In [29]:
population_2000_2020 = population_2000_2020.assign(age = lambda x: x['age'].str.extract('(\d+)'))
population_2000_2020

,date,state,age,german_male,german_female,german_total,foreigner_male,foreigner_female,foreigner_total,total_male,total_female,total
0,2000-12-31,Baden-Württemberg,1,52648,50202,102850,1748,1706,3454,54396,51908,106304
1,2000-12-31,Baden-Württemberg,1,48282,45273,93555,7981,7411,15392,56263,52684,108947
2,2000-12-31,Baden-Württemberg,2,49526,46826,96352,8128,7813,15941,57654,54639,112293
3,2000-12-31,Baden-Württemberg,3,51550,48941,100491,8509,8026,16535,60059,56967,117026
4,2000-12-31,Baden-Württemberg,4,50966,48167,99133,8063,7889,15952,59029,56056,115085
...,...,...,...,...,...,...,...,...,...,...,...,...
30906,2020-12-31,Thüringen,86,4426,7701,12127,7,11,18,4433,7712,12145
30907,2020-12-31,Thüringen,87,2973,5387,8360,9,8,17,2982,5395,8377
30908,2020-12-31,Thüringen,88,2468,4925,7393,11,11,22,2479,4936,7415
30909,2020-12-31,Thüringen,89,2079,4335,6414,11,3,14,2090,4338,6428


In [30]:
population_2000_2020['age'] = pd.to_numeric(population_2000_2020['age'])

In [31]:
population_2000_2020['date'] =  pd.to_datetime(population_2000_2020['date'])

In [32]:
population_2000_2020["country"]= "Germany"
population_2000_2020

,date,state,age,german_male,german_female,german_total,foreigner_male,foreigner_female,foreigner_total,total_male,total_female,total,country
0,2000-12-31,Baden-Württemberg,1,52648,50202,102850,1748,1706,3454,54396,51908,106304,Germany
1,2000-12-31,Baden-Württemberg,1,48282,45273,93555,7981,7411,15392,56263,52684,108947,Germany
2,2000-12-31,Baden-Württemberg,2,49526,46826,96352,8128,7813,15941,57654,54639,112293,Germany
3,2000-12-31,Baden-Württemberg,3,51550,48941,100491,8509,8026,16535,60059,56967,117026,Germany
4,2000-12-31,Baden-Württemberg,4,50966,48167,99133,8063,7889,15952,59029,56056,115085,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30906,2020-12-31,Thüringen,86,4426,7701,12127,7,11,18,4433,7712,12145,Germany
30907,2020-12-31,Thüringen,87,2973,5387,8360,9,8,17,2982,5395,8377,Germany
30908,2020-12-31,Thüringen,88,2468,4925,7393,11,11,22,2479,4936,7415,Germany
30909,2020-12-31,Thüringen,89,2079,4335,6414,11,3,14,2090,4338,6428,Germany


In [ ]:
from sql import engine
print(engine)

In [ ]:
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table
table_name = 'population_2000_2020'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This will take a minute or two...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        population_2000_2020.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        # schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None